In [ ]:
import calliope
import pandas as pd
from building import Building
import geopandas as gpd
import gc
import os
import matplotlib.pyplot as plt

In [ ]:
zone_df = gpd.read_file(r'C:\Users\wangy\OneDrive\ETHY2FW\IDP_Personal\CEA\2050 w3\inputs\building-geometry\zone.shp')
# iterate through the zone_df to create the building objects and store them in a list
building_list = []
# check if building's name is already stored in subfolder result_per_building; if not, add it to the list
for index, row in zone_df.iterrows():
    building_name = row['Name']
    if building_name+'.nc' not in os.listdir('./result_per_building_3'):
        building_list.append(building_name)
    else:
        print(building_name+' is already done')
    
# optimize the building one by one and store them in result_per_building as netcdf files
for building_string in building_list:
    building = Building(building_string, r'C:\Users\wangy\OneDrive\ETHY2FW\IDP_Personal\CEA\2050 w3')
    dict_timeseries_df = {'demand_el': building.app,
                          'demand_sh': building.sh,
                          'demand_dhw': building.dhw,
                          'demand_sc': building.sc,
                          'supply_PV': building.pv,
                          'supply_PVT_e': building.pvt_e_intensity,
                          'supply_PVT_h': building.pvt_h_relative_intensity,
                          'supply_SCFP': building.scfp_intensity
                          }
    building_specific_config: calliope.AttrDict = calliope.AttrDict.from_yaml('./data/technology/techs_plot8.yml')
    building_specific_config.set_key(key='locations.Building.available_area', value=building.area)
    building_sub_dict = building_specific_config['locations'].pop('Building')
    building_specific_config['locations'][building.name] = building_sub_dict
    model = calliope.Model(building_specific_config, timeseries_dataframes=dict_timeseries_df)
    model.run()
    # save to result_per_building, which is a subfolder of current working directory
    model.to_netcdf(path='./result_per_building_3/'+building.name+'.nc')
    # calculate how many percent of the buildings are done, including the current one and its name
    print('including '+building.name+', '+str(round(len(os.listdir('./result_per_building_2'))/len(zone_df.index)*100, 2))+'% of the buildings are done')
    
    del model
    # also delete the instance and the timeseries dataframes to save memory
    del building
    del dict_timeseries_df
    # clean up the rest of the memory
    gc.collect()

In [ ]:
# read all the netcdf files in the subfolder, for every building, find the capacity data of the optimal configuration for each building, and store them as a line in a dataframe
energy_cap_df = pd.DataFrame()
for file in os.listdir('./result_per_building_2'):
    model = calliope.read_netcdf('./result_per_building_2/'+file)
    # add the building name as a row index, and all energy_cap values as a row, and don't use append because it's deprecated
    # first define the column names
    if energy_cap_df.empty:
        energy_cap_df = pd.DataFrame(columns=model.get_formatted_array('energy_cap').to_pandas().columns)
    # then add the row
    energy_cap_df.loc[file[:-3]] = model.get_formatted_array('energy_cap').to_pandas().iloc[0]
    
    

In [ ]:
# plot the DHW_to_heat and wood_boiler_DHW fractions using df method
energy_cap_df['fraction_DHW_to_heat'] = energy_cap_df['DHW_to_heat'] / energy_cap_df['wood_boiler_DHW']
energy_cap_df.plot(y='fraction_DHW_to_heat', kind='hist', bins=100)


In [ ]:
print(building1.area)

In [ ]:
model = calliope.Model(building_specific_config, timeseries_dataframes=dict_timeseries_df)

In [ ]:

model.run()

In [ ]:
model.inputs.cost_purchase.to_pandas().dropna(axis=1)

In [ ]:
model.plot.summary()

In [ ]:
model.results.